<a href="https://colab.research.google.com/github/textnorms/date_text_norm/blob/master/T5_V12_En_relative_dates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Sun Jun 14 17:14:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! rm -rf date*
! git clone https://github.com/textnorms/date_text_norm.git
! cp -r date_text_norm/syntetic_data/ .

! pip install -q num2words transformers
! pip install -q transformers

Cloning into 'date_text_norm'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 294 (delta 48), reused 57 (delta 26), pack-reused 211
Receiving objects: 100% (294/294), 1.44 MiB | 3.33 MiB/s, done.
Resolving deltas: 100% (166/166), done.
     |████████████████████████████████| 102kB 5.2MB/s 
     |████████████████████████████████| 675kB 7.0MB/s 
     |████████████████████████████████| 890kB 23.0MB/s 
     |████████████████████████████████| 3.8MB 45.1MB/s 
     |████████████████████████████████| 1.1MB 57.7MB/s 


# Libs

### Choose Language

In [0]:
# LANGUAGE = 'pt'
LANGUAGE = 'en'

In [0]:
# Basics
import numpy as np
import pandas as pd
import random

# Synthetic data generator
from syntetic_data import DateTextGenerator
from syntetic_data import RelativeDateTextGenerator

# PyTorch
import torch 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Sklearn
from sklearn.model_selection import train_test_split

# Transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW

# Matplot lib
import matplotlib.pyplot as plt

### Deterministic experiments

In [5]:
manual_seed = 2357 # only primes, cuz I like
def deterministic(rep=True):
    if rep:
        np.random.seed(manual_seed)
        torch.manual_seed(manual_seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(manual_seed)
            torch.cuda.manual_seed_all(manual_seed)
        torch.backends.cudnn.enabled = False 
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        print(f'Deterministic experiment, seed: {manual_seed}')
    else:
        print('Random experiment')

deterministic()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Deterministic experiment, seed: 2357
Using device: cuda


# Config constants

In [6]:
# Model params
MODEL_SZ = 't5-small' # 't5-base'
TOK = T5Tokenizer.from_pretrained(MODEL_SZ)
MAX_LEN_SRC  = 48
MAX_LEN_TRGT = 12

# Train params
BATCH_SZ = 16
N_EPOCHS = 50
WINDOW   = 7

# Dataset

In [7]:
pd.set_option('display.max_rows',70)

print('Generating absolute and incomplete dates')
dates = DateTextGenerator(start_date='01/01/1921',
                          end_date='31/12/2120',
                          text_noise_rate=0.0,
                          language=LANGUAGE)

print('Generating relative dates')
rel_dates = RelativeDateTextGenerator(text_noise_rate=0.0,
                                       max_noise_occurences_per_sample=3,
                                       samples_per_method=15,
                                       language=LANGUAGE)

df = dates.generate_date_dataset()
df = df.append(rel_dates.generate_date_dataset(),ignore_index=True)

Generating absolute and incomplete dates
Generating relative dates


In [8]:
df.head()

,Input Pattern,Noise Type,Input,Target,Target Format
0,16,N/A,"01 january one thousand, nine hundred and twen...",01/01/1921,DD/MM/YYYY
1,23,N/A,02 - 1 - 1921,02/01/1921,DD/MM/YYYY
2,29,N/A,03 - jan - 1921,03/01/1921,DD/MM/YYYY
3,31,N/A,04 . 1 . 1921,04/01/1921,DD/MM/YYYY
4,8,N/A,five - 01 - 1921,05/01/1921,DD/MM/YYYY


In [9]:
df['Target Format'].value_counts()

DD/MM/YYYY    73049
MM/YYYY        7200
DD/MM          2562
Relative       1500
Name: Target Format, dtype: int64

In [10]:
df.loc[df['Target Format'] == 'MM/YYYY']

,Input Pattern,Noise Type,Input,Target,Target Format
75611,2,N/A,"jan of one thousand, nine hundred and twenty-one",01/1921,MM/YYYY
75612,30,N/A,1.1921,01/1921,MM/YYYY
75613,36,N/A,jan.1921,01/1921,MM/YYYY
75614,17,N/A,"02 one thousand, nine hundred and twenty-one",02/1921,MM/YYYY
75615,40,N/A,february/1921,02/1921,MM/YYYY
...,...,...,...,...,...
82806,7,N/A,"11 of two thousand, one hundred and twenty",11/2120,MM/YYYY
82807,10,N/A,november of 2120,11/2120,MM/YYYY
82808,8,N/A,12 - 2120,12/2120,MM/YYYY
82809,17,N/A,"12 two thousand, one hundred and twenty",12/2120,MM/YYYY


In [11]:
df.loc[df['Target Format'] == 'Relative']

,Input Pattern,Noise Type,Input,Target,Target Format
82811,17,N/A,during 1 year,1y,Relative
82812,2,N/A,one day ago,-1d,Relative
82813,6,N/A,one year ago,-1y,Relative
82814,3,N/A,1 month ago,-1m,Relative
82815,9,N/A,for 1 month,1m,Relative
...,...,...,...,...,...
84306,1,N/A,100 days ago,-100d,Relative
84307,9,N/A,for 100 months,100m,Relative
84308,4,N/A,one hundred months ago,-100m,Relative
84309,5,N/A,100 years ago,-100y,Relative


In [0]:
# Removing dates in the defined Target Format
df = df.loc[df['Target Format'] == 'Relative']

In [13]:
# Inspecting the relative dates dataset
df.head()

,Input Pattern,Noise Type,Input,Target,Target Format
82811,17,N/A,during 1 year,1y,Relative
82812,2,N/A,one day ago,-1d,Relative
82813,6,N/A,one year ago,-1y,Relative
82814,3,N/A,1 month ago,-1m,Relative
82815,9,N/A,for 1 month,1m,Relative


## Function to split the dataset

In [14]:
def split_data(df, test_size=0.2, verbose=True):
    l = list(set(df['Input Pattern'].values))
    num_test = int(len(l)*test_size)
    test_methods = [random.randint(1, len(l)) for _ in range(num_test)]
    print(test_methods)
    df_test = df[df['Input Pattern'].isin(test_methods)]
    print(df_test.shape)
    x_test = df_test.Input.values
    y_test = df_test.Target.values

    df_train = df[~df['Input Pattern'].isin(test_methods)]

    x_train, x_val, y_train, y_val = train_test_split(
        df_train.Input.values,
        df_train.Target.values,
        shuffle=True, 
        test_size=test_size,
        random_state=manual_seed
        )
    if verbose:
        print(f'Date types of test set: {test_methods} with len: {len(test_methods)}')
        print(f'x_train: {len(x_train)}  --  y_train: {len(y_train)}\n\
x_val:   {len(x_val)}  --  y_val:   {len(y_val)}\n\
x_test:  {len(x_test)}  --  y_test:  {len(y_test)}')

    return x_train, y_train, x_val, y_val, x_test, y_test

# creating sets
x_train, y_train, x_val, y_val, x_test, y_test = split_data(df, 
                                                            test_size=0.25, 
                                                            verbose=True)

[3, 15, 2, 11]
(339, 5)
Date types of test set: [3, 15, 2, 11] with len: 4
x_train: 870  --  y_train: 870
x_val:   291  --  y_val:   291
x_test:  339  --  y_test:  339


In [0]:
class DateDataset(Dataset):
    def __init__(self, data, label, tokenizer, source_max_length, target_max_length):
        self.tokenizer = tokenizer
        self.data = data
        self.label = label
        self.source_max_length = source_max_length
        self.target_max_length = target_max_length
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        source = self.data[idx]
        target = self.label[idx]

        source_tokenized = self.tokenizer.encode_plus(
            f'{source} {self.tokenizer.eos_token}',
            max_length=self.source_max_length,
            pad_to_max_length=True,
            return_tensors='pt')

        target_tokenized = self.tokenizer.encode_plus(
            f'{target} {self.tokenizer.eos_token}',
            max_length=self.target_max_length,
            pad_to_max_length=True,
            return_tensors='pt')

        source_token_ids = source_tokenized['input_ids'].squeeze()
        source_mask = source_tokenized['attention_mask'].squeeze()
        target_token_ids = target_tokenized['input_ids'].squeeze()
        
        return source_token_ids, source_mask, target_token_ids

## Checking the DateDataset class

In [16]:
dataset_debug = DateDataset(
    x_train, 
    y_train,
    TOK,
    MAX_LEN_SRC,
    MAX_LEN_TRGT,
    )

dataloader_checking = DataLoader(
    dataset_debug, 
    batch_size=1, 
    shuffle=True, 
    num_workers=0
    )

source_token_ids, source_mask, target_token_ids = next(iter(dataloader_checking))
print(f'source_token_ids:\n {source_token_ids} --- shape:{source_token_ids.shape}')
print(f'source_mask:\n {source_mask} --- shape:{source_mask.shape}')
print(f'target_token_ids:\n {target_token_ids} --- shape:{target_token_ids.shape}')

source_token_ids:
 tensor([[ 383, 4169,  477,    1,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]]) --- shape:torch.Size([1, 48])
source_mask:
 tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]) --- shape:torch.Size([1, 48])
target_token_ids:
 tensor([[668,  26,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0]]) --- shape:torch.Size([1, 12])


## Datasets e Dataloaders

In [17]:
# datasets
ds_debug = DateDataset(x_train[:BATCH_SZ], y_train[:BATCH_SZ], TOK, MAX_LEN_SRC, MAX_LEN_TRGT)
ds_train = DateDataset(x_train, y_train, TOK, MAX_LEN_SRC, MAX_LEN_TRGT)
ds_valid = DateDataset(x_val, y_val, TOK, MAX_LEN_SRC, MAX_LEN_TRGT)
ds_test  = DateDataset(x_test, y_test, TOK, MAX_LEN_SRC, MAX_LEN_TRGT)

print('Datasets len:')
print(f'len ds_debug: {len(ds_debug)}')
print(f'len ds_train: {len(ds_train)}')
print(f'len ds_valid: {len(ds_valid)}')
print(f'len ds_test:  {len(ds_test)}')

# dataloaders
dataloaders = {
    'debug': DataLoader(
         ds_debug,
         batch_size=BATCH_SZ,
         shuffle=True,
         num_workers=2,
         pin_memory=True),
    'train': DataLoader(
         ds_train,
         batch_size=BATCH_SZ,
         shuffle=True,
         num_workers=2,
         pin_memory=True),
    'valid': DataLoader(
         ds_valid,
         batch_size=BATCH_SZ,
         shuffle=False,
         num_workers=2,
         pin_memory=True),
    'test': DataLoader(
         ds_test,
         batch_size=BATCH_SZ,
         shuffle=False,
         num_workers=2,
         pin_memory=True),
               }
# sanity check
print('\nDataloaders len (in batch):')
dl_sizes = {x: len(dataloaders[x]) for x in dataloaders.keys()}; dl_sizes

Datasets len:
len ds_debug: 16
len ds_train: 870
len ds_valid: 291
len ds_test:  339

Dataloaders len (in batch):


{'debug': 1, 'test': 22, 'train': 55, 'valid': 19}

In [0]:
# testando o dataloader 
source_token_ids, source_mask, target_token_ids = next(iter(dataloaders['debug']))

In [0]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_SZ)
    
    def forward(self, token_ids, att_mask, labels):
        outputs = self.model.forward(
            input_ids=token_ids, 
            attention_mask=att_mask,
            lm_labels=labels
            )
        return outputs[0] # loss
    
    @torch.no_grad()    
    def generate(self, token_ids, att_mask, max_len_target):
        predict = self.model.generate(
            input_ids=token_ids, 
            attention_mask=att_mask,
            max_length=max_len_target
            )
        return predict
    
    @torch.no_grad()  
    def generate_example(self, text_input, tokenizer, max_len_source=MAX_LEN_SRC):

        self.model.eval()
        
        example_tokenized = tokenizer.encode_plus(
            f'{text_input} {tokenizer.eos_token}',
            max_length=max_len_source,
            pad_to_max_length=True,
            return_tensors='pt')
            
        example_token_ids = example_tokenized['input_ids']
        example_mask = example_tokenized['attention_mask']

        predicted_example = self.model.generate(
            input_ids=example_token_ids.to(device), 
            attention_mask=example_mask.to(device),
            max_length=MAX_LEN_TRGT
            )

        self.model.train()

        out_text = [tokenizer.decode(text) for text in predicted_example]
        
        return out_text

## Train and evaluation functions

In [0]:
# acc metric for text inputs
def acc_in_text(trues, preds): 
    acc = []
    for d in zip(trues, preds):
        if d[0] == d[1]:
            acc.append(1)
        else:
            acc.append(0)
    return acc # bool

def train(model, device, train_loader, optimizer):
    loss_train = []
    model.train()
    for source_token_ids, source_mask, target_token_ids in train_loader:
        optimizer.zero_grad()
        loss = model(
            source_token_ids.to(device), 
            source_mask.to(device), 
            target_token_ids.to(device)
            )
        
        loss_train.append(loss.item())
        loss.backward()
        optimizer.step()
    
    train_losses = sum(loss_train) / len(loss_train)
  
    return train_losses

def evaluate_fn(model, device, val_loader, max_len=MAX_LEN_TRGT):
    loss_val, all_acc, all_preds, all_trues = [], [], [], []
    model.eval()
    for source_token_ids, source_mask, target_token_ids in val_loader:
        predicted_ids = model.generate(
            source_token_ids.to(device), 
            source_mask.to(device),
            max_len
            )
        
        preds = [TOK.decode(t) for t in predicted_ids]
        trues = [TOK.decode(t) for t in target_token_ids]
        acc = acc_in_text(trues, preds)
        all_acc.extend(acc)
        all_trues.extend(trues)
        all_preds.extend(preds)
        
        # val loss   
        loss = model(
        source_token_ids.to(device), 
        source_mask.to(device), 
        target_token_ids.to(device)
        )
        loss_val.append(loss.item())
    
    val_losses = sum(loss_val) / len(loss_val)
    
    return val_losses, np.array(all_acc).mean(), all_trues, all_preds

# Overfit in one batch 
- dataloader debug

In [24]:
overfit = True

if overfit:

    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    deterministic() 

    model = Net().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    
    # -----------------------------------------------------------------------------
    start.record()
    for step in range(1, 1001):
        samp = random.randint(0, BATCH_SZ-WINDOW) # to show random trues and preds
        loss_t = train(model, device, dataloaders['debug'], optimizer)
        loss,acc, trues, preds = evaluate_fn(model, device, dataloaders['debug'])
        if step == 1:
            print(f'[Epoch: {step}/{1000}] |', end=' ')
            print(f'Train Loss: {loss_t:.3f} -- Acc: {acc:.3f}')
        if step % 100 == 0:
            print(f'[Epoch: {step}/{1000}] |', end=' ')
            print(f'Train Loss: {loss_t:.3f} -- Acc: {acc:.3f}')
            print(f'  Trues: {trues[samp:samp+WINDOW]}\n  Preds: {preds[samp:samp+WINDOW]}')
        if acc>= 0.99:
          print('The model has overfitted! Breaking the loop :)')
          break
    end.record()
    torch.cuda.synchronize()    
    # -----------------------------------------------------------------------------

    print(f'Training time: {start.elapsed_time(end)/1000/60 :.3f} min.')
    del model

Deterministic experiment, seed: 2357
[Epoch: 1/1000] | Train Loss: 13.511 -- Acc: 0.000
[Epoch: 100/1000] | Train Loss: 1.240 -- Acc: 0.000
  Trues: ['51d', '20y', '34d', '-65m', '38d', '-6m', '67m']
  Preds: ['', '', '', '', '', '', '']
[Epoch: 200/1000] | Train Loss: 0.196 -- Acc: 0.875
  Trues: ['71y', '76y', '20y', '5m', '-19y', '13d', '34d']
  Preds: ['71y', '76y', '20y', '5m', '190y', '13d', '34d']
The model has overfitted! Breking the loop :)
Training time: 1.752 min.


# Training 

In [26]:
try:
  del model
except:
  print('Model already erased, starting a new one!')
  
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
deterministic() 

model = Net().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# ---------------------------------------------------------------------------------
start.record()
for step in range(1, N_EPOCHS+1):
    samp = random.randint(0, BATCH_SZ-WINDOW) # to show random trues and preds
    loss_t = train(model, device, dataloaders['train'], optimizer)
    loss_v, acc, trues, preds = evaluate_fn(model, device, dataloaders['valid'])
    print(f'[Epoch: {step}/{N_EPOCHS}] |', end=' ')
    print(f'Train Loss: {loss_t:.3f} -- Valid Loss: {loss_v:.3f} -- Acc: {acc:.3f}')
    print(f'  Trues: {trues[samp:samp+WINDOW]}\n  Preds: {preds[samp:samp+WINDOW]}')

end.record()
torch.cuda.synchronize()    
# ---------------------------------------------------------------------------------

print(f'Training time: {start.elapsed_time(end)/1000/60 :.3f} min.')

Model already erased, starting a new one!
Deterministic experiment, seed: 2357
[Epoch: 1/50] | Train Loss: 4.631 -- Valid Loss: 1.452 -- Acc: 0.000
  Trues: ['-74m', '-53y', '92d', '-17y', '10m', '55d', '42y']
  Preds: ['', '', '', '', '', '', '']
[Epoch: 2/50] | Train Loss: 1.774 -- Valid Loss: 0.575 -- Acc: 0.024
  Trues: ['-17y', '10m', '55d', '42y', '31m', '55m', '4d']
  Preds: ['', '', '', '', '', '', '']
[Epoch: 3/50] | Train Loss: 0.797 -- Valid Loss: 0.186 -- Acc: 0.553
  Trues: ['-74m', '-53y', '92d', '-17y', '10m', '55d', '42y']
  Preds: ['74m', '53y', '92d', '17y', 'zehnm', '55d', '42y']
[Epoch: 4/50] | Train Loss: 0.347 -- Valid Loss: 0.109 -- Acc: 0.639
  Trues: ['84y', '-48m', '-74m', '-53y', '92d', '-17y', '10m']
  Preds: ['84y', '-48m', '74m', '53y', '92d', '17y', 'zehnm']
[Epoch: 5/50] | Train Loss: 0.236 -- Valid Loss: 0.074 -- Acc: 0.722
  Trues: ['42y', '31m', '55m', '4d', '3m', '59d', '-31m']
  Preds: ['42y', '31m', '55m', '4d', '3m', '59d', '-31m']
[Epoch: 6/50] |

KeyboardInterrupt: ignored

# Test

In [27]:
# ---------------------------------------------------------------------------------
start.record()

samp = random.randint(0, BATCH_SZ-WINDOW) # to show random trues and preds
loss, acc, trues, preds = evaluate_fn(model, device, dataloaders['test'])
print(f'Loss: {loss:.3f} -- Acc: {acc:.3f}')
print(f' Trues: {trues[samp:samp+WINDOW]}\n  Preds: {preds[samp:samp+WINDOW]}')

end.record()
torch.cuda.synchronize()    
# ---------------------------------------------------------------------------------

print(f'Test time: {start.elapsed_time(end)/1000/60 :.3f} min.')

Loss: 0.008 -- Acc: 0.988
 Trues: ['-2m', '2m', '3m', '3y', '-3m', '-3d', '-4m']
  Preds: ['-2m', '2m', '3m', '3y', '-3m', '-3d', '-4m']
Test time: 0.038 min.


# Evaluating types for a same date

Given a sample date, this section evaluates wich is the accuracy.




In [0]:
def evaluate_for_a_same_date(date,model=model,tokenizer=TOK,verbose=True):
  '''
    Given a specific date, returns the accuracy in all evalueated types.
    Also prints results per sample.
  '''

  results = []
  
  examples = datas.generate_demo(date=date)

  for x,target in zip(examples['Generated Text'],examples['Origin Sample']):

    prediction = model.generate_example(x,TOK)[0]

    results.append(prediction == target)

    if verbose:
      print(f'Entrada: {x} -- Target: {target} --- Previsto: {prediction} --- {prediction == target}')

  if verbose:
    print(f'Total accuracy: {np.mean(results)}')

  return np.mean(results)


In [0]:
'''
  A date in the century with more dates occuring
'''
evaluate_for_a_same_date('11/07/1988')

In [0]:
'''
  A date in a century with less dates occurring, but that is inside the
  generated dataset
'''
evaluate_for_a_same_date('20/12/2015')

In [0]:
'''
  Evaluating for a date in a century out of the training range gives the worst
  acc possible. (0)
'''
evaluate_for_a_same_date('25/12/2141')

In [0]:
'''
  An earlier date than the beggining of the generated dataset
'''
evaluate_for_a_same_date('27/05/1920')

# Accuracy in dataset dates

## Inside dataset

Dates into the interval that was used to built the synthetic dataset used for test and eval.

In [0]:
accs = []
dates = []

sampled_test = random.sample(list(df['Target'].values),50)

print('acc test set: ',sampled_test)

for date_sample in sampled_test:
  accs.append(evaluate_for_a_same_date(date_sample,verbose=False))
  dates.append(date_sample)
  

In [0]:
plt.plot(dates,accs)
plt.xticks(rotation=45)
plt.grid()
plt.ylim([0.8, 1.01])
plt.ylabel('Average accuracy on 45 formats')
plt.xlabel('Canonical target')
plt.title('Average accuracy on dates inside synthetic training dataset')
print('Average of average accuracies: ',np.mean(accs))

## Below dataset

Dates lower than the synthetic dataset used for test and eval.

In [0]:
accs = []
dates = []

new_dataset = DateTextGenerator('01/01/1900','31/12/1920')

new_df = new_dataset.generate_date_dataset()
sampled_test = random.sample(list(new_df['Target'].values),20)

print('acc test set: ',sampled_test)

for date_sample in sampled_test:
  accs.append(evaluate_for_a_same_date(date_sample,verbose=False))
  dates.append(date_sample)

In [0]:
plt.plot(dates,accs)
plt.xticks(rotation=45)
plt.grid()
plt.ylim([-0.01, 1.01])
plt.ylabel('Average accuracy on 45 formats')
plt.xlabel('Canonical target')
plt.title('Average accuracy on dates below synthetic training dataset')
print('Average of average accuracies: ',np.mean(accs))

## Above dataset

Dates greater than the synthetic dataset used for test and eval.

In [0]:
accs = []
dates = []

new_dataset = DateTextGenerator('01/01/2121','31/12/2140')


new_df = new_dataset.generate_date_dataset()
sampled_test = random.sample(list(new_df['Target'].values),20)

print('acc test set: ',sampled_test)

for date_sample in sampled_test:
  accs.append(evaluate_for_a_same_date(date_sample,verbose=False))
  dates.append(date_sample)

In [0]:
plt.plot(dates,accs)
plt.xticks(rotation=45)
plt.grid()
plt.ylim([-0.01, 1.01])
plt.ylabel('Average accuracy on 45 formats')
plt.xlabel('Canonical target')
plt.title('Average accuracy on dates above synthetic training dataset')
print('Average of average accuracies: ',np.mean(accs))

# The End